In [172]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import bs4
import pandas as pd
driver = webdriver.Chrome()
driver.get('https://th.tripadvisor.com/Attraction_Review-g303915-d6669435-Reviews-Wat_Pho_Chai-Nong_Khai_Nong_Khai_Province.html')

review_data = pd.DataFrame(columns = ['name', 'city', 'score','title','comment', 'date'])

In [173]:
def getmonth(mtxt):
    if(mtxt == "มกราคม"):
        return "01"
    elif(mtxt == "กุมภาพันธ์"):
        return "02"
    elif(mtxt == "มีนาคม"):
        return "03"
    elif(mtxt == "เมษายน"):
        return "04"
    elif(mtxt == "พฤษภาคม"):
        return "05"
    elif(mtxt == "มิถุนายน"):
        return "06"
    elif(mtxt == "กรกฎาคม"):
        return "07"
    elif(mtxt == "สิงหาคม"):
        return "08"
    elif(mtxt == "กันยายน"):
        return "09"
    elif(mtxt == "ตุลาคม"):
        return "10"
    elif(mtxt == "พฤศจิกายน"):
        return "11"
    else:
        return "12"

def loaddata():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    from datetime import datetime

    all_review_score = soup.find_all('svg',{'class':'RWYkj d H0'})
    all_review_comment = soup.find_all('span',{'class':'NejBf'})
    all_review_date = soup.find_all('div',{'class':'WlYyy diXIH cspKb bQCoY'})
    all_review_user = soup.find_all('div',{'class':'cjhIj'})

    all_review_score_list = []
    all_review_comment_list = []
    all_review_comment_header_list = []
    all_review_date_list = []
    all_review_user_list = []
    all_review_user_city_list = []



    for rev in all_review_user:    
        if(rev.span.a != None):
            all_review_user_list.append(rev.span.a.text)
            all_review_user_city_list.append(rev.div.text.split()[0].replace(',','') if ',' in rev.div.text.split()[0] else rev.div.text.split()[0])

    for review in all_review_date:
        date = review.text.split()[1] if len(review.text.split()[1]) == 2 else "0" + review.text.split()[1]
        year = review.text.split()[4][2:4]
        month = getmonth(review.text.split()[2])
        datetime_txt = date + "/" + month + "/" + year
        review_date = datetime.strptime(datetime_txt, '%d/%m/%y')
        all_review_date_list.append(datetime_txt)

    for review in all_review_score :
        if(len(review['aria-label'].split()) > 0):
            all_review_score_list.append(float(review['aria-label'].split()[1]))
    i = 0
    while i < len(all_review_comment) :    
        all_review_comment_header_list.append(all_review_comment[i].text)
        all_review_comment_list.append(all_review_comment[i+1].text)
        i+=2
    df = pd.DataFrame([all_review_user_list, all_review_user_city_list, all_review_score_list, all_review_comment_header_list, all_review_comment_list, all_review_date_list])
    df = df.transpose()
    df.columns = ['name', 'city', 'score','title','comment', 'date']
    df.dropna(subset = ["name"], inplace=True)
    return df

def addtodf(review_data, df):
    review_data = pd.concat([review_data, df], ignore_index=True)
    len(review_data)
    return review_data

df = loaddata()
addtodf(review_data, df)

,name,city,score,title,comment,date
0,paitoon,กรุงเทพมหานคร,5.0,วัดโพธิ์ เป็นวัดพระอารามหลวง,วัดโพธิ์ชัยเป็นสถานที่ศักดิ์ของหนองคายและประเท...,22/07/20
1,narasri99,เมืองนนทบุรี,5.0,วัดหลวงพ่อพระใส องค์พระงดงาม,วัดติดถนนใหญ่เดินทางไปไหว้พระทำบุญได้สะดวก บร...,30/12/19
2,Nix B,กรุงเทพมหานคร,4.0,วัดที่คนมาหนองคายต้องมากราบไหว้,เป็นวัดทีชาวหนองคายนับถือ โดยมีหลวงพ่อใสที่ศัก...,27/11/19
3,Nujjaree23,สูงเนิน,5.0,กราบพระใส,วัดโพธิ์ชัยนั้นเป็นพระอารามหลวง ตั้งอยู่ที่ถนน...,20/09/19
4,Pukimi,กรุงเทพมหานคร,5.0,กราบหลวงพ่อพระใส,กราบสักการะหลวงพ่อพระใส หลวงพ่อศักดิ์สิทธิ์คู...,30/07/19
5,taweewun,เมืองพะเยา,4.0,วัดหลวงพ่อพระใส,เป็นวัดที่ติดกับ บขส.หนองคาย ประตูรั้วด้านข้าง...,26/04/19
6,นันทิตา,เมืองหนองคาย,4.0,ศรัทธา,มาเที่ยวหนองคายต้องมาขอพรที่นี่หลวงพ่อพระใส สิ...,23/04/19
7,fondoodee,กรุงเทพมหานคร,5.0,ไหว้พระใส,มาหนองคายต้องมาไหว้วัดพระใส สวยงาม ศักดิ์สิทธิ...,29/01/19
8,gik15,Bangkok,4.0,วัดโพธิ์ชัย,กราบขอพรหลวงพ่อพระใส วัดโพธิ์ชัย จังหวัดหนองคา...,05/09/18
9,PhatteenanK,เมืองอุดรธานี,5.0,วัดโพธิ์ชัย,วัดพระใสเป็นวัดที่มีความเชื่อว่าลูกหลานพญานาค ...,02/02/18


In [183]:
button_next = driver.find_elements(By.CLASS_NAME, 'cCnaz')
ActionChains(driver).click(button_next.pop()).perform()

In [ ]:
df = loaddata()
print(len(df))
review_data = addtodf(review_data, df)
print(len(review_data))

In [151]:
review_data.to_excel(r'results\review1.xlsx')